In [1]:
import numpy as np
import random
import plotly.express as px

x = np.linspace(0, 10, 100)
y = x + np.random.normal(size=x.size)

A = np.array([[np.sum(x**2), np.sum(x)], [np.sum(x), x.size]])
b = np.array([np.sum(x*y), np.sum(y)])
coef = np.linalg.solve(A, b)


fig = px.scatter(x=x, y=y)
fig.add_scatter(x=x, y=coef[0]*x + coef[1], mode='lines')
fig.show()

In [2]:
# quadratic

x = np.linspace(0, 10, 100)
y = 2*x**2 + np.random.normal(size=x.size)*20

A = np.array([[np.sum(x**4), np.sum(x**3), np.sum(x**2)],
              [np.sum(x**3), np.sum(x**2), np.sum(x)],
              [np.sum(x**2), np.sum(x), x.size]])

b = np.array([np.sum(x**2*y), np.sum(x*y), np.sum(y)])
coef = np.linalg.solve(A, b)

fig = px.scatter(x=x, y=y)
fig.add_scatter(x=x, y=coef[0]*x**2 + coef[1]*x + coef[2], mode='lines')
fig.show()

## Příklad úlohy

Máme pět výrobců, tři mezisklady a čtyři odběratele. Úkolem je minimalizovat celkové náklady na transport a skladování produktů, respektující kapacitní limity a požadavky všech účastníků.

    Producenti: Vyprodukují produkty, které musí být převezeny do meziskladů.
    Mezisklady: Přijímají produkty od producentů, skladují je (za účtování nákladů na skladování) a posílají je dál k odběratelům.
    Odběratelé: Mají určité požadavky na objem produktů, které musí být splněny.

### Data

    Kapacity producentů (produkty): [20, 30, 25, 10, 15]
    Požadavky odběratelů (produkty): [10, 20, 30, 10]
    Kapacity meziskladů (produkty): [30, 40, 30]
    Náklady na skladování (za jednotku produktu v každém sklade): [2, 3, 2.5]
    Náklady na přepravu:
        Producenti do meziskladů: [[1, 2, 3], [2, 1, 3], [1, 3, 1], [4, 3, 2], [2, 3, 2]]
        Mezisklady k odběratelům: [[2, 3, 1, 4], [3, 2, 2, 1], [1, 5, 2, 3]]

### Matematický model

    Proměnné:
        xijxij​: množství produktu převezeného od producenta ii do meziskladu jj
        yjkyjk​: množství produktu převezeného od meziskladu jj k odběrateli kk

    Cílová funkce: Minimizace celkových nákladů na transport a skladování.

    Omezení:
        Produkce každého producenta je rovna dodávkám do meziskladů.
        Požadavky každého odběratele musí být splněny z dodávek meziskladů.
        Kapacita meziskladů musí být respektována.

## Matematický model úlohy minimalizace nákladů na transport a skladování

Máme systém se pěti výrobců, třemi mezisklady a čtyřmi odběrateli. Úkolem je minimalizovat celkové náklady na transport a skladování zboží.

### Proměnné
- $x_{ij}$: Množství produktu převezeného od producenta $i$ do meziskladu $j$.
- $y_{jk}$: Množství produktu převezeného od meziskladu $j$ k odběrateli $k$.

### Cílová funkce
Minimizujeme celkové náklady na transport a skladování:
$$
\text{Minimizovat} \quad Z = \sum_{i=1}^5 \sum_{j=1}^3 x_{ij} c_{ij} + \sum_{j=1}^3 \sum_{k=1}^4 y_{jk} d_{jk} + \sum_{i=1}^5 \sum_{j=1}^3 x_{ij} s_j
$$
kde:
- $c_{ij}$ jsou náklady na přepravu od producenta $i$ do meziskladu $j$,
- $d_{jk}$ jsou náklady na přepravu od meziskladu $j$ k odběrateli $k$,
- $s_j$ jsou náklady na skladování v meziskladu $j$.

### Omezení
1. **Bilance výroby a přepravy**:
   $$
   \sum_{j=1}^3 x_{ij} = p_i \quad \forall i \in \{1, 2, 3, 4, 5\}
   $$
   kde $p_i$ jsou kapacity jednotlivých producentů.

2. **Bilance požadavků odběratelů**:
   $$
   \sum_{j=1}^3 y_{jk} = q_k \quad \forall k \in \{1, 2, 3, 4\}
   $$
   kde $q_k$ jsou požadované objemy odběratelů.

3. **Kapacitní omezení meziskladů**:
   $$
   \sum_{i=1}^5 x_{ij} \leq r_j \quad \forall j \in \{1, 2, 3\}
   $$
   kde $r_j$ jsou kapacity meziskladů.

4. **Non-negativita**:
   $$
   x_{ij} \geq 0, \quad y_{jk} \geq 0
   $$

In [7]:
import pulp as pl

# Indexy
producers = range(5)
warehouses = range(3)
customers = range(4)

# Data
supply = [20, 30, 25, 10, 15]
demand = [10, 20, 30, 10]
warehouse_capacity = [30, 40, 30]
storage_costs = [2, 3, 2.5]
cost_producer_to_warehouse = [[1, 2, 3], [2, 1, 3], [1, 3, 1], [4, 3, 2], [2, 3, 2]]
cost_warehouse_to_customer = [[2, 3, 1, 4], [3, 2, 2, 1], [1, 5, 2, 3]]

# Model
model = pl.LpProblem("Two_stage_distribution", pl.LpMinimize)

# Proměnné
x = pl.LpVariable.dicts("Shipments_Prod_Warehouse", (producers, warehouses), lowBound=0)
y = pl.LpVariable.dicts("Shipments_Warehouse_Cust", (warehouses, customers), lowBound=0)

# Cílová funkce
model += (
    pl.lpSum(x[i][j] * cost_producer_to_warehouse[i][j] for i in producers for j in warehouses) +
    pl.lpSum(y[j][k] * cost_warehouse_to_customer[j][k] for j in warehouses for k in customers) +
    pl.lpSum(x[i][j] * storage_costs[j] for i in producers for j in warehouses)
)

# Omezení
for i in producers:
    model += pl.lpSum(x[i][j] for j in warehouses) == supply[i]

for k in customers:
    model += pl.lpSum(y[j][k] for j in warehouses) == demand[k]

for j in warehouses:
    model += pl.lpSum(x[i][j] for i in producers) <= warehouse_capacity[j]

# Řešení (no message)
model.solve(pl.PULP_CBC_CMD(msg=0))
print("Status:", pl.LpStatus[model.status])

# Výsledky
for i in producers:
    for j in warehouses:
        if x[i][j].varValue > 0:
            print(f"Producer {i} -> Warehouse {j}: {x[i][j].varValue}")

for j in warehouses:
    for k in customers:
        if y[j][k].varValue > 0:
            print(f"Warehouse {j} -> Customer {k}: {y[j][k].varValue}")

Status: Optimal
Producer 0 -> Warehouse 0: 10.0
Producer 0 -> Warehouse 1: 10.0
Producer 1 -> Warehouse 1: 30.0
Producer 2 -> Warehouse 0: 20.0
Producer 2 -> Warehouse 2: 5.0
Producer 3 -> Warehouse 2: 10.0
Producer 4 -> Warehouse 2: 15.0
Warehouse 0 -> Customer 2: 30.0
Warehouse 1 -> Customer 1: 20.0
Warehouse 1 -> Customer 3: 10.0
Warehouse 2 -> Customer 0: 10.0


In [12]:
import pulp as pl

# Příkladový model
model = pl.LpProblem("Test_Multiple_Solutions", pl.LpMinimize)
x = pl.LpVariable("x", lowBound=0)
y = pl.LpVariable("y", lowBound=0)
model += x + y
model += x + y >= 10, "constraint1"

# Řešení
model.solve(pl.PULP_CBC_CMD(msg=0))

# Kontrola duálních hodnot
for name, c in list(model.constraints.items()):
    print(f"Duální hodnota pro {name} je: {c.pi}")

# Zjistíme, jestli x nebo y jsou na svém limitu
print(f"Hodnota x: {x.varValue}, Na limitu: {'Ano' if x.varValue == x.lowBound else 'Ne'}")
print(f"Hodnota y: {y.varValue}, Na limitu: {'Ano' if y.varValue == y.lowBound else 'Ne'}")


Duální hodnota pro constraint1 je: 1.0
Hodnota x: 10.0, Na limitu: Ne
Hodnota y: 0.0, Na limitu: Ano
